Some good transfer style tutorials:

https://medium.com/tensorflow/neural-style-transfer-creating-art-with-deep-learning-using-tf-keras-and-eager-execution-7d541ac31398

https://towardsdatascience.com/neural-style-transfer-tutorial-part-1-f5cd3315fa7f

https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/15_Style_Transfer.ipynb![image.png](attachment:image.png)

Below is mainly based on the TF tutorial.

Github for 3D style transfer: https://github.com/hiroharu-kato/style_transfer_3d/blob/master/style_transfer_3d/main.py

**Issues/Tasks:**
1. need to scale the image to 0-1?
2. need to subtract the mean style when calculate style loss?
3. add back print and save functions for 3D
4. OOM!!

## Environment Setup

In [100]:
import os, time, itertools
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure


# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import PIL.Image

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.random.seed(0)
tf.set_random_seed(0)

tf.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'1.12.0'

## Import Data

In [101]:
# import data
import env
from data.thingi10k import Thingi10k
from data.stl import save_vectors_as_stl
from data.stl import plot_mesh

thingi = Thingi10k.init10()
n_samples = len(thingi)
n_samples

10

In [102]:
# some models are quite large and cause this architecture to hit a memory error
# also tensors cannot be greater than 2 GB!
# to calculate size in bytes of tensor, take size of architecture layer * vector length * 4

#n_input = 90000  # multiple of 9!! vertices come in 3! any less and you're cutting off vital info
n_input = thingi.max_length()

print('n_samples: {}'.format(n_samples))
print('n_input: {}'.format(n_input))
size_bytes = n_input * 100 * 4
print('size: {} bytes'.format(size_bytes))
size_gb = size_bytes / 1024 / 1024 / 1024
print('size: {:.2f} gb'.format(size_bytes / 1024 / 1024 / 1024))
assert size_gb < 2

n_samples: 10
n_input: 276048
size: 110419200 bytes
size: 0.10 gb


In [103]:
output_dir = os.path.join('_output', 'style_transfer')
os.makedirs(output_dir, exist_ok=True)
graphs_dir = os.path.join(output_dir, 'graphs')
os.makedirs(graphs_dir, exist_ok=True)
model_dir = os.path.join(output_dir, 'model')
os.makedirs(model_dir, exist_ok=True)

## VGG-16 Model

In [104]:
import vgg16
# # download the data if not loaded, it's 550MB!!
vgg16.data_dir = 'vgg16/'
vgg16.maybe_download()

Data has apparently already been downloaded and unpacked.


In [105]:
## image helper functions

def load_image(filename, max_size=None):
    image = PIL.Image.open(filename)

    if max_size is not None:
        # Calculate the appropriate rescale-factor for
        # ensuring a max height and width, while keeping
        # the proportion between them.
        factor = max_size / np.max(image.size)
    
        # Scale the image's height and width.
        size = np.array(image.size) * factor

        # The size is now floating-point because it was scaled.
        # But PIL requires the size to be integers.
        size = size.astype(int)

        # Resize the image.
        image = image.resize(size, PIL.Image.LANCZOS)

    # Convert to numpy floating-point array.
    return np.float32(image)


def save_image(image, filename):
    # Ensure the pixel-values are between 0 and 255.
    image = np.clip(image, 0.0, 255.0)
    
    # Convert to bytes.
    image = image.astype(np.uint8)
    
    # Write the image-file in jpeg-format.
    with open(filename, 'wb') as file:
        PIL.Image.fromarray(image).save(file, 'jpeg')
        
def plot_image_big(image):
    # Ensure the pixel-values are between 0 and 255.
    image = np.clip(image, 0.0, 255.0)

    # Convert pixels to bytes.
    image = image.astype(np.uint8)

    # Convert to a PIL-image and display it.
    display(PIL.Image.fromarray(image))
    
def plot_images(content_image, style_image, mixed_image):
    # Create figure with sub-plots.
    fig, axes = plt.subplots(1, 3, figsize=(10, 10))

    # Adjust vertical spacing.
    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    # Use interpolation to smooth pixels?
    smooth = True
    
    # Interpolation type.
    if smooth:
        interpolation = 'sinc'
    else:
        interpolation = 'nearest'

    # Plot the content-image.
    # Note that the pixel-values are normalized to
    # the [0.0, 1.0] range by dividing with 255.
    ax = axes.flat[0]
    ax.imshow(content_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Content")

    # Plot the mixed-image.
    ax = axes.flat[1]
    ax.imshow(mixed_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Mixed")

    # Plot the style-image
    ax = axes.flat[2]
    ax.imshow(style_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Style")

    # Remove ticks from all the plots.
    for ax in axes.flat:
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
# import chainer.functions as cf
# def extract_style_feature(self, images, masks=None):

#         xp = self.xp

#         mean = xp.array([103.939, 116.779, 123.68], 'float32')  # BGR

#         images = images[:, ::-1] * 255 - mean[None, :, None, None]

#         features = self.vgg16(images, layers=['conv1_2', 'conv2_2', 'conv3_3', 'conv4_3']).values()

#         if masks is None:

#             masks = xp.ones((images.shape[0], images.shape[2], images.shape[3]))



#         style_features = []

#         for feature in features:

#             scale = masks.shape[-1] / feature.shape[-1]

#             m = cf.average_pooling_2d(masks[:, None, :, :], scale, scale).data

#             dim = feature.shape[1]



#             m = m.reshape((m.shape[0], -1))

#             f2 = feature.transpose((0, 2, 3, 1))

#             f2 = f2.reshape((f2.shape[0], -1, f2.shape[-1]))

#             f2 *= xp.sqrt(m)[:, :, None]

#             f2 = cf.batch_matmul(f2.transpose((0, 2, 1)), f2)

#             f2 /= dim * m.sum(axis=1)[:, None, None]

#             style_features.append(f2)



#         return style_features

In [106]:
## Loss functions

def mean_squared_error(a, b):
    return tf.reduce_mean(tf.square(a - b))

def create_content_loss(session, model, content_image, layer_ids):
    """
    Create the loss-function for the content-image.
    
    Parameters:
    session: An open TensorFlow session for running the model's graph.
    model: The model, e.g. an instance of the VGG16-class.
    content_image: Numpy float array with the content-image.
    layer_ids: List of integer id's for the layers to use in the model.
    """
    
    # Create a feed-dict with the content-image.
    feed_dict = model.create_feed_dict(image=content_image)

    # Get references to the tensors for the given layers.
    layers = model.get_layer_tensors(layer_ids)

    # Calculate the output values of those layers when
    # feeding the content-image to the model.
    values = session.run(layers, feed_dict=feed_dict)

    # Set the model's graph as the default so we can add
    # computational nodes to it. It is not always clear
    # when this is necessary in TensorFlow, but if you
    # want to re-use this code then it may be necessary.
    with model.graph.as_default():
        # Initialize an empty list of loss-functions.
        layer_losses = []
    
        # For each layer and its corresponding values
        # for the content-image.
        for value, layer in zip(values, layers):
            # These are the values that are calculated
            # for this layer in the model when inputting
            # the content-image. Wrap it to ensure it
            # is a const - although this may be done
            # automatically by TensorFlow.
            value_const = tf.constant(value)

            # The loss-function for this layer is the
            # Mean Squared Error between the layer-values
            # when inputting the content- and mixed-images.
            # Note that the mixed-image is not calculated
            # yet, we are merely creating the operations
            # for calculating the MSE between those two.
            loss = mean_squared_error(layer, value_const)

            # Add the loss-function for this layer to the
            # list of loss-functions.
            layer_losses.append(loss)

        # The combined loss for all layers is just the average.
        # The loss-functions could be weighted differently for
        # each layer. You can try it and see what happens.
        total_loss = tf.reduce_mean(layer_losses)
        
    return total_loss

def gram_matrix(tensor):
    shape = tensor.get_shape()
       
#     mean = np.array([103.939, 116.779, 123.68], 'float32')  # BGR

    # Get the number of feature channels for the input tensor,
    # which is assumed to be from a convolutional layer with 4-dim.
    num_channels = int(shape[3])


    # Reshape the tensor so it is a 2-dim matrix. This essentially
    # flattens the contents of each feature-channel.
    matrix = tf.reshape(tensor, shape=[-1, num_channels])
    
    # Calculate the Gram-matrix as the matrix-product of
    # the 2-dim matrix with itself. This calculates the
    # dot-products of all combinations of the feature-channels.
    gram = tf.matmul(tf.transpose(matrix), matrix)

    return gram

def create_style_loss(session, model, style_image, layer_ids):
    """
    Create the loss-function for the style-image.
    
    Parameters:
    session: An open TensorFlow session for running the model's graph.
    model: The model, e.g. an instance of the VGG16-class.
    style_image: Numpy float array with the style-image.
    layer_ids: List of integer id's for the layers to use in the model.
    """

    # Create a feed-dict with the style-image.
    feed_dict = model.create_feed_dict(image=style_image) # 4-d dimension

    # Get references to the tensors for the given layers.
    layers = model.get_layer_tensors(layer_ids)

    # Set the model's graph as the default so we can add
    # computational nodes to it. It is not always clear
    # when this is necessary in TensorFlow, but if you
    # want to re-use this code then it may be necessary.
    with model.graph.as_default():
        # Construct the TensorFlow-operations for calculating
        # the Gram-matrices for each of the layers.
        gram_layers = [gram_matrix(layer) for layer in layers]

        # Calculate the values of those Gram-matrices when
        # feeding the style-image to the model.
        values = session.run(gram_layers, feed_dict=feed_dict)

        # Initialize an empty list of loss-functions.
        layer_losses = []
    
        # For each Gram-matrix layer and its corresponding values.
        for value, gram_layer in zip(values, gram_layers):
            # These are the Gram-matrix values that are calculated
            # for this layer in the model when inputting the
            # style-image. Wrap it to ensure it is a const,
            # although this may be done automatically by TensorFlow.
            value_const = tf.constant(value)

            # The loss-function for this layer is the
            # Mean Squared Error between the Gram-matrix values
            # for the content- and mixed-images.
            # Note that the mixed-image is not calculated
            # yet, we are merely creating the operations
            # for calculating the MSE between those two.
            loss = mean_squared_error(gram_layer, value_const)

            # Add the loss-function for this layer to the
            # list of loss-functions.
            layer_losses.append(loss)

        # The combined loss for all layers is just the average.
        # The loss-functions could be weighted differently for
        # each layer. You can try it and see what happens.
        total_loss = tf.reduce_mean(layer_losses)
        
    return total_loss

def create_denoise_loss(model):
    loss = tf.reduce_sum(tf.abs(model.input[:,1:,:,:] - model.input[:,:-1,:,:])) + \
           tf.reduce_sum(tf.abs(model.input[:,:,1:,:] - model.input[:,:,:-1,:]))
#     loss = tf.reduce_sum(tf.abs(model.input[:,1:] - model.input[:,:-1]))

    return loss

In [107]:
## style-transfer algorithm
def style_transfer(content_image, style_image,
                   content_layer_ids, style_layer_ids,
                   weight_content=1.5, weight_style=10.0,
                   weight_denoise=0.3,
                   num_iterations=120, step_size=10.0):
    """
    Use gradient descent to find an image that minimizes the
    loss-functions of the content-layers and style-layers. This
    should result in a mixed-image that resembles the contours
    of the content-image, and resembles the colours and textures
    of the style-image.
    
    Parameters:
    content_image: Numpy 3-dim float-array with the content-image.
    style_image: Numpy 3-dim float-array with the style-image.
    content_layer_ids: List of integers identifying the content-layers.
    style_layer_ids: List of integers identifying the style-layers.
    weight_content: Weight for the content-loss-function.
    weight_style: Weight for the style-loss-function.
    weight_denoise: Weight for the denoising-loss-function.
    num_iterations: Number of optimization iterations to perform.
    step_size: Step-size for the gradient in each iteration.
    """

    # Create an instance of the VGG16-model. This is done
    # in each call of this function, because we will add
    # operations to the graph so it can grow very large
    # and run out of RAM if we keep using the same instance.
    model = vgg16.VGG16()

    # Create a TensorFlow-session.
    session = tf.InteractiveSession(graph=model.graph)

    # Print the names of the content-layers.
    print("Content layers:")
    print(model.get_layer_names(content_layer_ids))
    print()

    # Print the names of the style-layers.
    print("Style layers:")
    print(model.get_layer_names(style_layer_ids))
    print()

    # Create the loss-function for the content-layers and -image.
    loss_content = create_content_loss(session=session,
                                       model=model,
                                       content_image=content_image,
                                       layer_ids=content_layer_ids)

    # Create the loss-function for the style-layers and -image.
    loss_style = create_style_loss(session=session,
                                   model=model,
                                   style_image=style_image,
                                   layer_ids=style_layer_ids)    

    # Create the loss-function for the denoising of the mixed-image.
    loss_denoise = create_denoise_loss(model)

    # Create TensorFlow variables for adjusting the values of
    # the loss-functions. This is explained below.
    adj_content = tf.Variable(1e-10, name='adj_content')
    adj_style = tf.Variable(1e-10, name='adj_style')
    adj_denoise = tf.Variable(1e-10, name='adj_denoise')

    # Initialize the adjustment values for the loss-functions.
    session.run([adj_content.initializer,
                 adj_style.initializer,
                 adj_denoise.initializer])

    # Create TensorFlow operations for updating the adjustment values.
    # These are basically just the reciprocal values of the
    # loss-functions, with a small value 1e-10 added to avoid the
    # possibility of division by zero.
    update_adj_content = adj_content.assign(1.0 / (loss_content + 1e-10))
    update_adj_style = adj_style.assign(1.0 / (loss_style + 1e-10))
    update_adj_denoise = adj_denoise.assign(1.0 / (loss_denoise + 1e-10))

    # This is the weighted loss-function that we will minimize
    # below in order to generate the mixed-image.
    # Because we multiply the loss-values with their reciprocal
    # adjustment values, we can use relative weights for the
    # loss-functions that are easier to select, as they are
    # independent of the exact choice of style- and content-layers.
    loss_combined = weight_content * adj_content * loss_content + \
                    weight_style * adj_style * loss_style + \
                    weight_denoise * adj_denoise * loss_denoise

    # Use TensorFlow to get the mathematical function for the
    # gradient of the combined loss-function with regard to
    # the input image.
    gradient = tf.gradients(loss_combined, model.input)

    # List of tensors that we will run in each optimization iteration.
    run_list = [gradient, update_adj_content, update_adj_style, \
                update_adj_denoise]

    # The mixed-image is initialized with random noise.
    # It is the same size as the content-image.
    mixed_image = np.random.rand(*content_image.shape) + 128 # voxel should be 0 or 1, and no color channel!!

    for i in range(num_iterations):
        # Create a feed-dict with the mixed-image.
        feed_dict = model.create_feed_dict(image=mixed_image)

        # Use TensorFlow to calculate the value of the
        # gradient, as well as updating the adjustment values.
        grad, adj_content_val, adj_style_val, adj_denoise_val \
        = session.run(run_list, feed_dict=feed_dict)

        # Reduce the dimensionality of the gradient.
        grad = np.squeeze(grad)

        # Scale the step-size according to the gradient-values.
        step_size_scaled = step_size / (np.std(grad) + 1e-8)

        # Update the image by following the gradient.
        mixed_image -= grad * step_size_scaled

        # Ensure the image has valid pixel-values between 0 and 255.
        mixed_image = np.clip(mixed_image, 0.0, 255.0)

        # Print a little progress-indicator.
        print(". ", end="")

        # Display status once every 10 iterations, and the last.
        if (i % 10 == 0) or (i == num_iterations - 1):
            print()
            print("Iteration:", i)

            # Print adjustment weights for loss-functions.
            msg = "Weight Adj. for Content: {0:.2e}, Style: {1:.2e}, Denoise: {2:.2e}"
            print(msg.format(adj_content_val, adj_style_val, adj_denoise_val))

#             # Plot the content-, style- and mixed-images.
#             plot_images(content_image=content_image,
#                         style_image=style_image,
#                         mixed_image=mixed_image)
#             try:
            
#                 dim = 256 ##?
# #                 vox_vectors = np.reshape(mixed_image, (dim, dim, dim))
#                 vox_vectors = np.reshape(mixed_image, (dim, dim, dim))

#                 ckpt_dir = os.path.join(model_dir, 'epoch_{}'.format(epoch))
#                 os.makedirs(ckpt_dir, exist_ok=True)

#                 np_path = os.path.join(ckpt_dir, 'epoch_{}.npy'.format(epoch))
#                 np.save(np_path, mixed_image)

#                 # Use marching cubes to obtain the surface mesh
#                 verts, faces, normals, values = measure.marching_cubes_lewiner(vox_vectors, 0)
#                 save_vectors_as_stl(verts[faces], os.path.join(ckpt_dir, 'epoch_{}.stl'.format(epoch)))
#                 plot_mesh(verts[faces])
#             except Exception as exc:
#                 print('Model generation failed for this epoch')
#                 print(exc)
#     print()
#     print("Final image:")
#     plot_image_big(mixed_image)
    

    # Close the TensorFlow session to release its resources.
    session.close()
    
    # Return the mixed-image.
    return mixed_image

In [108]:
## example
# content_filename = 'images/willy_wonka_old.jpg'
# content_image = load_image(content_filename, max_size=None)

## assuming both content and style input are 3D
for batch_num, batch in enumerate(thingi.voxels_batchmaker(batch_size=1)):
    content_image = np.reshape(batch[0],(256,256,256))
    break
style_filename = 'images/style7.jpg'
style_image = load_image(style_filename, max_size=300)
style_image = np.expand_dims(style_image, axis=0)
print("content shape: ", content_image.shape)
print("style shape: ", style_image.shape)

content shape:  (256, 256, 256)
style shape:  (1, 298, 300, 3)


In [109]:
# use average style color for content input as background?
mean = style_image.mean((0, 1, 2))
# print(mean)
content_image = np.expand_dims(content_image, axis=3)
content_image = content_image + mean[None, None, None, :]
print("content shape: ", content_image.shape)
print("style shape: ", style_image.shape)

content shape:  (256, 256, 256, 3)
style shape:  (1, 298, 300, 3)


In [113]:
content_layer_ids = [4]

# The VGG16-model has 13 convolutional layers.
# This selects all those layers as the style-layers.
# This is somewhat slow to optimize.
# style_layer_ids = list(range(13))

# You can also select a sub-set of the layers, e.g. like this:
style_layer_ids = [1, 2, 3, 4]

In [114]:
%%time
img = style_transfer(content_image=content_image,
                     style_image=style_image,
                     content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,
                     weight_content=1.5,
                     weight_style=10.0,
                     weight_denoise=0.3,
                     num_iterations=2,
                     step_size=10.0)

Content layers:
['conv3_1/conv3_1']

Style layers:
['conv1_2/conv1_2', 'conv2_1/conv2_1', 'conv2_2/conv2_2', 'conv3_1/conv3_1']



ResourceExhaustedError: OOM when allocating tensor with shape[256,256,256,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node conv1_1/Conv2D (defined at C:\Users\cynth\desktop\3d-form\notebooks\vgg16.py:116)  = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](concat, conv1_1/filter)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'conv1_1/Conv2D', defined at:
  File "C:\Users\cynth\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\cynth\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\cynth\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Us

In [112]:
# # Directory to store the downloaded data.
# data_dir = "vgg16/"

# # File containing the TensorFlow graph definition. (Downloaded)
# path_graph_def = "vgg16.tfmodel"

# # Name of the tensor for feeding the input image.
# tensor_name_input_image = "images:0"

# # Names of the tensors for the dropout random-values..
# tensor_name_dropout = 'dropout/random_uniform:0'
# tensor_name_dropout1 = 'dropout_1/random_uniform:0'

# # Names for the convolutional layers in the model for use in Style Transfer.
# x_layer_names = ['conv1_1/conv1_1', 'conv1_2/conv1_2',
#                'conv2_1/conv2_1', 'conv2_2/conv2_2',
#                'conv3_1/conv3_1', 'conv3_2/conv3_2', 'conv3_3/conv3_3',
#                'conv4_1/conv4_1', 'conv4_2/conv4_2', 'conv4_3/conv4_3',
#                'conv5_1/conv5_1', 'conv5_2/conv5_2', 'conv5_3/conv5_3']


# # Now load the model from file. The way TensorFlow
# # does this is confusing and requires several steps.

# # Create a new TensorFlow computational graph.
# x_graph = tf.Graph()

# # Set the new graph as the default.
# with x_graph.as_default():

#     # TensorFlow graphs are saved to disk as so-called Protocol Buffers
#     # aka. proto-bufs which is a file-format that works on multiple
#     # platforms. In this case it is saved as a binary file.

#     # Open the graph-def file for binary reading.
#     path = os.path.join(data_dir, path_graph_def)
#     with tf.gfile.FastGFile(path, 'rb') as file:
#         # The graph-def is a saved copy of a TensorFlow graph.
#         # First we need to create an empty graph-def.
#         graph_def = tf.GraphDef()

#         # Then we load the proto-buf file into the graph-def.
#         graph_def.ParseFromString(file.read())

#         # Finally we import the graph-def to the default TensorFlow graph.
#         tf.import_graph_def(graph_def, name='')

#         # Now self.graph holds the VGG16 model from the proto-buf file.

#     # Get a reference to the tensor for inputting images to the graph.
# #     x_input = x_graph.get_tensor_by_name(x_tensor_name_input_image)

#     # Get references to the tensors for the commonly used layers.
#     x_layer_tensors = [x_graph.get_tensor_by_name(name + ":0") for name in x_layer_names]

# x_layer_tensors